In [16]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime
from selenium import webdriver
import time
import requests
import re

In [2]:
file_name = "hong_df.csv"

In [3]:
try:
    hong_df = pd.read_csv(file_name, encoding="utf-8")
    hong_db = hong_df.to_dict()
    values = []
    for i in hong_db.values():
        v = list(i.values())
        values.append(v)
    for i,(k,v) in enumerate(hong_db.items()):
        hong_db[k] = values[i]
    print("{}파일을 성공적으로 불러왔습니다.".format(file_name))
except:
    print("불러올수 있는 {}파일이 없습니다.".format(file_name))
    options = webdriver.ChromeOptions()
    # options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    # 혹은 options.add_argument("--disable-gpu")

    wd = webdriver.Chrome('/home/ai106/PycharmProjects/pythonProject/heatmap_fastfood/chromedriver', 
                         options=options)
    stores = []
    for i in range(33):
        hong_url = 'https://www.hongruizhen.com/store/store.php?page=' + str(i+1)
        wd.get(hong_url)
        time.sleep(0.3)

        html = wd.page_source
        soup_hong = BeautifulSoup(html, 'html.parser')
        store_list = soup_hong.select('#contents > div > div.store_wr > div.store_cont > div > div > div.store_list')
        for store in store_list:
            store_info = store.select("div > ul > li > div.store_detail > p:nth-child(4)")
            for store_addr in store_info:
                print(store_addr.get_text()[6:])
                stores.append(store_addr.get_text()[6:])

    trash = ['전북 정읍시 충청로 59',
    '전북 군산시 월명로 24',
    '경기 수원시 권선구 서수원로 577']

    for i in trash:
        stores.remove(i)

    def getLatLng(address):
        result = ""
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
        rest_api_key = '58871f76e7240525a7f5d85c8b83ef1c'
        header = {'Authorization': 'KakaoAK ' + rest_api_key}
        r = requests.get(url, headers=header)
        if r.status_code == 200:
            result_address = r.json()["documents"][0]["address"]
            result = result_address["y"], result_address["x"]
        else:
            result = "ERROR[" + str(r.status_code) + "]"
        return result


    def getKakaoMapHtml(address_latlng):
        javascript_key = "bf0ed730bee63cd68c5a342ba9bbca54"
        result = ""
        result = result + "<div id='map' style='width:300px;height:200px;display:inline-block;'></div>" + "\n"
        result = result + "<script type='text/javascript' src='//dapi.kakao.com/v2/maps/sdk.js?appkey=" + javascript_key + "'></script>" + "\n"
        result = result + "<script>" + "\n"
        result = result + "    var container = document.getElementById('map'); " + "\n"
        result = result + "    var options = {" + "\n"
        result = result + "           center: new kakao.maps.LatLng(" + address_latlng[0] + ", " + address_latlng[1] + ")," + "\n"
        result = result + "           level: 3" + "\n"
        result = result + "    }; " + "\n"
        result = result + "    var map = new kakao.maps.Map(container, options); " + "\n"
        # 검색한 좌표의 마커 생성을 위해 추가
        result = result + "    var markerPosition  = new kakao.maps.LatLng(" + address_latlng[0] + ", " + address_latlng[1] + ");  " + "\n"
        result = result + "    var marker = new kakao.maps.Marker({position: markerPosition}); " + "\n"
        result = result + "    marker.setMap(map); " + "\n"
        result = result + "</script>" + "\n"
        return result

    latlngs = []
    hong_db = {}
    for addr in stores:
        print("\n", addr)
        # 카카오 REST API로 좌표 구하기
        address_latlng = getLatLng(addr)
        # 좌표로 지도 첨부 HTML 생성
        if str(address_latlng).find("ERROR") < 0:
            map_html = getKakaoMapHtml(address_latlng)
            html = BeautifulSoup(map_html, "html.parser")
            script = html.select("script")
            script = script[1].string
            latlng = re.findall("\d+", script)[:4]
            latlng = (latlng[0] + "." + latlng[1], latlng[2] + "." + latlng[3])
            print(latlng)
            latlngs.append(latlng)
            hong_db[addr] = latlng
        else:
            print("[ERROR]getLatLng")

    hong_df = pd.DataFrame(hong_db)
    hong_df.to_csv("hong_df.csv", encoding='utf-8', index=False)       

hong_df.csv파일을 성공적으로 불러왔습니다.


In [4]:
search_word = ["서울", "경기"]

In [5]:
hong_db

{'대구광역시 북구 연경지묘로 8': [35.9427734353765, 128.614999112889],
 '서울특별시 영등포구 영등포로 103': [37.5220662013046, 126.893196256907],
 '경기도 성남시 분당구 중앙공원로31번길 42': [37.383133453219, 127.125492233427],
 '충청북도 충주시 5산단1길 11': [37.0082011703369, 127.948895768335],
 '서울 동작구 대방동23길 44': [37.5041467532881, 126.92292810712],
 '경기도 수원시 봉영로 1606': [37.2538686061108, 127.074491002062],
 '강원도 춘천시 김유정로 1864': [37.8530387312842, 127.735392131478],
 '충북 청주시 청원구 오창읍 2산단3로 16': [36.7363059929718, 127.454677439831],
 '충북 충주시 중앙탑면 원앙9길 15': [37.0157412993212, 127.825918770774],
 '충남 보령시 중앙로 64': [36.3502781297562, 126.597262359996],
 '충남 공주시 번영1로 80': [36.4737050776796, 127.136504437672],
 '대전 유성구 엑스포로 481': [36.3992613341852, 127.40455442338],
 '충청남도 천안시 서북구 불당26로 122': [36.8197610766889, 127.10778351439],
 '충남 천안시 동남구 청수11로 18': [36.7826136053183, 127.154481525307],
 '대전 유성구 노은로 178': [36.3755909307344, 127.318507795507],
 '충북 제천시 풍양로15길 7-1': [37.1379255796078, 128.211593944754],
 '충북 충주시 갱고개로 176': [36.98665027528

In [6]:
def create_ex(search_word, db):
    addr = []
    lat_lngs = []
    for k, v in db.items():
        if search_word in k:
            lat_lngs.append(v)
            addr.append(k)

    ex_ = []
    for x in zip(*lat_lngs):
        ex_.append(list(x))

    ex2 = {}
    ex2['Latitude'] = ex_[0]
    ex2['Longitude'] = ex_[1]

    ex2 = pd.DataFrame(ex2)
    ex2['address'] = addr
    ex2['name'] = "홍루이젠"
    ex2['count'] = 1
       
    return ex2

In [7]:
ex_list = []
for word in search_word:
    ex = create_ex(word, hong_db)
    ex_list.append(ex)

In [47]:
ex_list

[     Latitude   Longitude                address  name  count
 0   37.522066  126.893196    서울특별시 영등포구 영등포로 103  홍루이젠      1
 1   37.504147  126.922928       서울 동작구 대방동23길 44  홍루이젠      1
 2   37.529082  126.921889         서울 영등포구 은행로 29  홍루이젠      1
 3   37.525649  127.053943        서울 강남구 도산대로 549  홍루이젠      1
 4   37.488024  127.101844      서울시 강남구 밤고개로1길 10  홍루이젠      1
 5   37.466806  127.101327   서울 강남구 강남구 헌릉로569길 3  홍루이젠      1
 6   37.508811  126.961442         서울 동작구 흑석로 115  홍루이젠      1
 7   37.481425  126.880394     서울 금천구 가산디지털1로 181  홍루이젠      1
 8   37.502498  127.046381      서울 강남구 테헤란로52길 16  홍루이젠      1
 9   37.450853  126.909674         서울 금천구 금하로 717  홍루이젠      1
 10  37.566590  127.024082     서울특별시 성동구 왕십리로 410  홍루이젠      1
 11  37.532814  126.900128         서울 영등포구 당산로205  홍루이젠      1
 12  37.652912  127.045587       서울 도봉구 노해로63길 67  홍루이젠      1
 13  37.509462  127.123466          서울 송파구 마천로 22  홍루이젠      1
 14  37.654538  127.063033         서울 노원구 노해로 494  홍루이젠

In [56]:
fig = go.Figure(go.Densitymapbox(lat=ex_list[0].Latitude, lon=ex_list[0].Longitude, 
                                 radius=10, colorscale=px.colors.sequential.Sunset))
if len(ex_list) > 1:
    for i in range(len(ex_list)-1):
        fig.add_trace(go.Densitymapbox(lat=ex_list[i+1].Latitude, lon=ex_list[1].Longitude, 
                                         radius=10, colorscale=px.colors.sequential.YlGn))

fig.update_layout(title=dict({"text":"홍루이젠", "font_size":15}))
fig.update_layout(mapbox_style="carto-positron", mapbox_center=dict(lat=35.6, lon=127))
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0}, mapbox=dict(zoom=5.5))
fig.update_traces()

fig.show()

# fig = px.density_mapbox(ex_list[0], lat='Latitude', lon='Longitude', z='count', radius=10,
#                         hover_name='name', hover_data=['address'],
#                         center=dict(lat=35.6, lon=127), zoom=5.5,
#                         mapbox_style="carto-positron", color_continuous_scale=px.colors.sequential.Sunset)

# fig.add_trace(px.density_mapbox(ex_list[1], lat='Latitude', lon='Longitude', z='count', radius=10,
#                         hover_name='name', hover_data=['address'], 
#                             color_continuous_scale=px.colors.sequential.YlGn))
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [10]:
fig = px.colors.sequential.swatches_continuous()
fig.show()

In [11]:
dir(px.colors.qualitative)

['Alphabet',
 'Alphabet_r',
 'Antique',
 'Antique_r',
 'Bold',
 'Bold_r',
 'D3',
 'D3_r',
 'Dark2',
 'Dark24',
 'Dark24_r',
 'Dark2_r',
 'G10',
 'G10_r',
 'Light24',
 'Light24_r',
 'Pastel',
 'Pastel1',
 'Pastel1_r',
 'Pastel2',
 'Pastel2_r',
 'Pastel_r',
 'Plotly',
 'Plotly_r',
 'Prism',
 'Prism_r',
 'Safe',
 'Safe_r',
 'Set1',
 'Set1_r',
 'Set2',
 'Set2_r',
 'Set3',
 'Set3_r',
 'T10',
 'T10_r',
 'Vivid',
 'Vivid_r',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_cols',
 '_contents',
 '_k',
 '_swatches',
 'swatches']

In [12]:
from PyQt5 import QtCore, QtWidgets, QtWebEngineWidgets
import plotly.express as px

class Widget(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.button = QtWidgets.QPushButton('Plot', self)
        self.browser = QtWebEngineWidgets.QWebEngineView(self)
        vlayout = QtWidgets.QVBoxLayout(self)
        vlayout.addWidget(self.button, alignment=QtCore.Qt.AlignHCenter)
        vlayout.addWidget(self.browser)
        self.button.clicked.connect(self.show_graph)
        self.resize(1000,800)
        
    def show_graph(self):
        #컬러스케일 보여주는 코드
#         fig = px.colors.sequential.swatches_continuous()
#         fig.show()
        fig = px.density_mapbox(ex2, lat='Latitude', lon='Longitude', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain", color_continuous_scale=px.colors.qualitative.Vivid)
        self.browser.setHtml(fig.to_html(include_plotlyjs='cdn'))
        
if __name__ == "__main__":
    app = QtWidgets.QApplication([])
    widget = Widget()
    widget.show()
    app.exec()

NameError: name 'ex2' is not defined

In [13]:
import plotly.express as px
country = px.data.gapminder()
fig = px.scatter_geo(country, locations="iso_alpha", 
                     color="continent",
                     hover_name="country", size="pop",
                     animation_frame="year",
#                      projection="natural earth"
                     projection="orthographic"
                     )
                     
fig.show()

In [14]:
country

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716
